In [1]:
import pandas as pd

data = pd.read_excel("data/Australian layers/AusTraits.xlsx")

In [ ]:
print(data.species.str.split("_").str[0].nunique())

foo


In [1]:
import torch
from transformers import AutoTokenizer, AutoModel
from PIL import Image
import numpy as np
import os

# load the model and tokenizer
path = "ByteDance/Sa2VA-4B"
model = AutoModel.from_pretrained(
    path,
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    use_flash_attn=True,
    trust_remote_code=True).eval().cuda()
tokenizer = AutoTokenizer.from_pretrained(path, trust_remote_code=True, use_fast=False)

# for image chat
image_path = "/home/smonaco/newzeland-plants/australian_imgs/Acacia_acradenia/166570371.jpg"
PROMPT = """
TASK
You will look only for FRUITS of the plant species in the image and output their COLOUR(S). 
Ignore flowers, leaves, stems, bark, background, containers, and artifacts.

SPECIES TEXT
Flower description (to ignore): {flower_description}
Fruit description (to match): {fruit_description}

COLOUR LABEL SET
black, blue, brown, green, red, white, yellow, purple, orange, grey

OUTPUT REQUIREMENTS
- Output only ONE line in this exact format (no extra words):
  <label>            e.g., "red"
  or
  <label>, <label>   e.g., "green, red"  (dominant first; max 2 colours)
- If no fruit is visible, or fruit is too small/hidden/ambiguous to be confident, output exactly: None

RULES
1) Detect candidate fruit regions; ignore anything matching only the flower description.
2) The region must be consistent with the provided fruit description (shape/arrangement/texture/position). If not, output None.
3) Determine the perceived surface colour(s) of the visible fruit(s).
   - If multiple fruits: choose the dominant overall colour; include a second colour only if it clearly covers >=25% of visible fruit area.
   - Ripening/bi-colour fruit: order by

"""

text_prompts = PROMPT.format(
    fruit_description=tmp['Acacia'],
    flower_description="Golden cylindrical flower spikes made of dense, tiny spherical florets",
    )
image = Image.open(image_path).convert('RGB')
input_dict = {
    'image': image,
    'text': text_prompts,
    'past_text': '',
    'mask_prompts': None,
    'tokenizer': tokenizer,
    }
return_dict = model.predict_forward(**input_dict)
answer = return_dict["prediction"] # the text format answer

# for image chat with segmentation output
# image_path = "/PATH/TO/IMAGE"
# text_prompts = "<image>Could you please give me a brief description of the image? Please respond with interleaved segmentation masks for the corresponding parts of the answer."
# image = Image.open(image_path).convert('RGB')
# input_dict = {
#     'image': image,
#     'text': text_prompts,
#     'past_text': '',
#     'mask_prompts': None,
#     'tokenizer': tokenizer,
#     }
# return_dict = model.predict_forward(**input_dict)
# answer = return_dict["prediction"] # the text format answer
# masks = return_dict['prediction_masks']  # segmentation masks, list(np.array(1, h, w), ...)

Encountered exception while importing flash_attn: No module named 'flash_attn'


ImportError: This modeling file requires the following packages that were not found in your environment: flash_attn. Run `pip install flash_attn`

In [2]:
import os
import requests
import pandas as pd
from tqdm.auto import tqdm
from io import BytesIO
plants = pd.read_excel("data/Australian layers/AusTraits.xlsx")
plant_names = plants.species.str.replace("_", " ")
from huggingface_hub import login

os.environ['HF_TOKEN'] = 'REDACTED_HF_TOKEN'

In [5]:
', '.join(plant_names)

'Abrotanella nivigena, Abrotanella scapigera, Acacia abbreviata, Acacia acinacea, Acacia aciphylla, Acacia acoma, Acacia acradenia, Acacia acuminata, Acacia adnata, Acacia adoxa, Acacia adsurgens, Acacia adunca, Acacia aestivalis, Acacia alata, Acacia alexandri, Acacia alpina, Acacia amblygona, Acacia ammobia, Acacia ampliceps, Acacia anaticeps, Acacia anceps, Acacia ancistrocarpa, Acacia andrewsii, Acacia aneura, Acacia anfractuosa, Acacia angusta, Acacia anthochaera, Acacia aphanoclada, Acacia aphylla, Acacia aprepta, Acacia araneosa, Acacia argutifolia, Acacia argyraea, Acacia arida, Acacia armitii, Acacia arrecta, Acacia ashbyae, Acacia aspera, Acacia assimilis, Acacia atkinsiana, Acacia attenuata, Acacia aulacocarpa, Acacia aulacophylla, Acacia auricoma, Acacia auriculiformis, Acacia auronitens, Acacia ausfeldii, Acacia ayersiana, Acacia baeuerlenii, Acacia baileyana, Acacia bakeri, Acacia balsamea, Acacia bancroftiorum, Acacia barattensis, Acacia barringtonensis, Acacia basedowii

In [27]:
PROMPT = """Looking at this image, write the colour of the fruit in the following format: 

- <colour> (e.g., brown, red, etc.) 
- <colour, colour> (e.g., brown green, green red, black blue, etc.) 
The available colours are: black, blue, brown, green, red, white, yellow, purple, orange, grey.
Answer 'None' if the image contains only flowers.
Answer 'None' if no fruit is visible in the image."""
species_list = plant_names.tolist()

In [28]:

# iNaturalist API helpers
INAT_BASE = "https://api.inaturalist.org/v1"

def fetch_observations_for_species(species_name, per_page=50, max_pages=4, photos_only=True):
    """Fetch a list of observations for a given species_name from iNaturalist."""
    observations = []
    for page in range(1, max_pages + 1):
        params = {
            "taxon_name": species_name,
            "page": page,
            "per_page": per_page,
            "order_by": "created_at",
            "order": "desc",
            "photos": "true" if photos_only else "false",
        }
        r = requests.get(f"{INAT_BASE}/observations", params=params, timeout=30)
        r.raise_for_status()
        data = r.json()
        if not data.get("results"):
            break
        observations.extend(data["results"])
        if len(data["results"]) < per_page:
            break
    return observations

def observation_likely_has_fruit(obs):
    """Attempt to detect whether the observation has a fruit annotation or mention.
    This is a heuristic: checks controlled annotations and free-text fields for 'fruit' keywords.
    """
    return True
    # check annotations
    if obs.get("description"):
        if "fruit" in obs["description"].lower():
            return True
    if obs.get("description"):
        if "fruits" in str(obs["comments"]).lower():
            return True
    
    if any("fruit" in tag.lower() for tag in obs.get("tags", [])):
        print('++++ found')
        return True
    return False

def download_photo(photo):
    """Download photo bytes from observation_photos item returned by iNaturalist.
    photo is expected to contain a 'photo' dict with a 'url' key (e.g. .../medium.jpg).
    """
    url = photo.get("photo", {}).get("url")
    if not url:
        return None
    # prefer 'original' size if available by replacing size token
    for sz in ("medium", "small", "square", "original"):
        if f"/{sz}." in url:
            # try to request original
            url_orig = url.replace(f"/{sz}.", "/original.")
            try:
                r = requests.get(url_orig, timeout=30)
                if r.status_code == 200:
                    return r.content
            except Exception:
                pass
            break
    # fallback to the provided url
    r = requests.get(url, timeout=30)
    r.raise_for_status()
    return r.content

In [29]:
import torch
from PIL import Image
from transformers import AutoModelForCausalLM, AutoModelForVision2Seq

class FruitColorVLM:
    """
    Analyze fruit color using a lightweight vision-language model.
    Using MobileVLM or similar lightweight VLM for local inference.
    """
    
    def __init__(self, model_name: str = "moondream/moondream3-preview", prompt: str = PROMPT, device: str = None):
        """
        Initialize the VLM for fruit color analysis.
        
        Args:
            model_name: VLM model name (Phi-3.5-vision is lightweight and efficient)
            device: Device to run inference on
        """
        self.device = device or ('cuda' if torch.cuda.is_available() else 'cpu')
        self.model_name = model_name
        self.model = None
        self.processor = None
        self.prompt = prompt
        
    def _init_model(self):
        """Lazy initialization of the vision-language model."""
        if self.model is not None:
            return
        
        print(f"Loading {self.model_name} vision-language model...")

        self.model = AutoModelForCausalLM.from_pretrained(
            "vikhyatk/moondream2",
            trust_remote_code=True,
            device_map="cuda", # "cuda" on Nvidia GPUs
        ).to(torch.bfloat16)
        self.settings = {"temperature": 0.5, "max_tokens": 768, "top_p": 0.3}
        
        
        
        print(f"VLM loaded on {self.device}")

    
    def get_fruit_color(self, img):
        """
        Extract fruit color from image using VLM.
        
        Args:
            image_path: Path to the image file
            
        Returns:
            Dict with 'color' and 'raw_response'
        """
        self._init_model()
        
        try:
            image = Image.open(BytesIO(img)).convert('RGB')
            
            answer = self.model.query(
                image,
                self.prompt,
                settings=self.settings
            )    
            if isinstance(answer, dict):
                answer = answer['answer']
            # Clean up the response (remove prompt if echoed)
            if self.prompt in answer:
                answer = answer.replace(self.prompt, "").strip()
            
            # Extract color from response
            color = self._parse_color(answer)
            
            return {
                'color': color,
                'raw_response': answer
            }
            
        except Exception as e:
            print(f"Error: {e}")
            return {
                'color': 'unknown',
                'raw_response': f'Error: {str(e)}'
            }
    
    def _parse_color(self, response: str) -> str:
        """
        Parse color from VLM response.
        
        Args:
            response: Raw text response from VLM
            
        Returns:
            Extracted color string
        """
        if response is None or response.strip() == "" or response.lower() == "none":
            return 'None'
            
        response_lower = response.lower()
        
        # Common fruit colors
        colors = [
            'red', 'green', 'yellow', 'orange', 'purple', 'blue',
            'brown', 'black', 'white', 'pink', 'burgundy', 'crimson',
            'golden', 'amber', 'violet', 'indigo', 'maroon', 'tan'
        ]
        
        # Check for color words
        found_colors = [c for c in colors if c in response_lower]
        
        if found_colors:
            return found_colors[0]  # Return first found color
        
        # Return full response if no specific color found
        return 'None'


In [30]:
rows = []
seen_photos = set()
vlm = FruitColorVLM()
per_species_limit = 500  # max observations per species to fetch

for i, species in tqdm(enumerate(species_list), total=len(species_list)):
    obs = fetch_observations_for_species(species, per_page=50, max_pages=(per_species_limit // 50) or 1)

    for o in tqdm(obs, desc=f"Observations for {species}", leave=False):
        if not observation_likely_has_fruit(o):
            # still consider passing through VLM if you want stricter filtering,
            # but this keeps API calls down by discarding obvious non-fruit obs.
            continue
        for op in o.get("observation_photos", []):
            photo_dict = op.get("photo") or op
            photo_id = photo_dict.get("id") or op.get("id")
            if not photo_id or photo_id in seen_photos:
                continue
            seen_photos.add(photo_id)
            try:
                img = download_photo(op)
            except Exception as e:
                # skip problematic downloads
                continue
            vlm_answer = vlm.get_fruit_color(img)
            
            rows.append({
                "image_id": photo_id,
                "species": species,
                "tags": o.get("tags", []),
            } | vlm_answer)

df = pd.DataFrame(rows, columns=["image_id", "species", "fruit_colour"])



  0%|          | 0/851 [00:00<?, ?it/s]

Observations for Abrotanella nivigena:   0%|          | 0/4 [00:00<?, ?it/s]

Loading moondream/moondream3-preview vision-language model...
VLM loaded on cuda


Observations for Abrotanella scapigera:   0%|          | 0/12 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [31]:
pd.DataFrame(rows)

,image_id,species,tags,color,raw_response
0,448558030,Abrotanella nivigena,[],None,None
1,448558044,Abrotanella nivigena,[],None,None
2,345435373,Abrotanella nivigena,[],None,None
3,345435397,Abrotanella nivigena,[],None,None
4,175791027,Abrotanella nivigena,[],brown,brown
5,75623410,Abrotanella nivigena,[Flower],None,None
6,75623415,Abrotanella nivigena,[Flower],None,None
7,75623418,Abrotanella nivigena,[Flower],None,None
8,472363775,Abrotanella scapigera,[],None,None
9,472363812,Abrotanella scapigera,[],None,None


In [ ]:
obs_df[['tags', 'description',]]

,quality_grade,annotations,identifications_most_agree,species_guess,tags,flags,description,taxon,outlinks,ofvs,preferences,geojson,faves
0,research,[],True,Abrotanella scapigera,[],[],,"{'is_active': True, 'ancestry': '48460/47126/2...","[{'source': 'Atlas of Living Australia', 'url'...",[],{'prefers_community_taxon': None},"{'type': 'Point', 'coordinates': [146.57061666...",[]
1,research,[],True,Abrotanella scapigera,[],[],None,"{'is_active': True, 'ancestry': '48460/47126/2...","[{'source': 'Atlas of Living Australia', 'url'...",[],{'prefers_community_taxon': None},"{'type': 'Point', 'coordinates': [145.62088582...",[]
2,needs_id,[],False,Abrotanella scapigera,[],[],Few plants seen on small south-facing rock led...,"{'is_active': True, 'ancestry': '48460/47126/2...","[{'source': 'Atlas of Living Australia', 'url'...",[],{'prefers_community_taxon': None},"{'type': 'Point', 'coordinates': [145.90448151...",[]
3,needs_id,[],False,Abrotanella scapigera,[],[],Locally abundant multi-tufted herb growing in ...,"{'is_active': True, 'ancestry': '48460/47126/2...","[{'source': 'Atlas of Living Australia', 'url'...",[],{'prefers_community_taxon': None},"{'type': 'Point', 'coordinates': [145.97404906...",[]
4,needs_id,[],False,Abrotanella scapigera,[],[],None,"{'is_active': True, 'ancestry': '48460/47126/2...","[{'source': 'Atlas of Living Australia', 'url'...",[],{'prefers_community_taxon': None},"{'type': 'Point', 'coordinates': [145.94901993...",[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
439,research,[],True,Acacia adoxa,[],[],None,"{'is_active': True, 'ancestry': '48460/47126/2...","[{'source': 'Atlas of Living Australia', 'url'...",[],{'prefers_community_taxon': None},"{'type': 'Point', 'coordinates': [118.73546448...",[]
440,research,[{'uuid': 'a96fa470-665a-4450-9d82-cd38e064851...,True,Acacia adoxa,[],[],None,"{'is_active': True, 'ancestry': '48460/47126/2...","[{'source': 'Atlas of Living Australia', 'url'...",[],{'prefers_community_taxon': None},"{'type': 'Point', 'coordinates': [118.26941403...",[]
441,research,[],True,Acacia adoxa,[],[],None,"{'is_active': True, 'ancestry': '48460/47126/2...","[{'source': 'Atlas of Living Australia', 'url'...",[],{'prefers_community_taxon': None},"{'type': 'Point', 'coordinates': [122.20074947...",[]
442,research,[],True,Acacia adoxa,[],[],None,"{'is_active': True, 'ancestry': '48460/47126/2...","[{'source': 'Atlas of Living Australia', 'url'...",[],{'prefers_community_taxon': None},"{'type': 'Point', 'coordinates': [118.4080647,...",[]


In [27]:
df[0]['annotations']

[{'uuid': '7846fbab-1cfc-4ee6-b254-d69e834b6587',
  'controlled_attribute_id': 12,
  'controlled_value_id': 13,
  'concatenated_attr_val': '12|13',
  'user_id': 3587396,
  'votes': [],
  'user': {'id': 3587396,
   'login': 'jackiemiles',
   'spam': False,
   'suspended': False,
   'created_at': '2020-09-10T06:15:02+00:00',
   'login_autocomplete': 'jackiemiles',
   'login_exact': 'jackiemiles',
   'name': '',
   'name_autocomplete': '',
   'orcid': None,
   'icon': 'https://static.inaturalist.org/attachments/users/icons/3587396/thumb.jpg?1603334730',
   'observations_count': 7460,
   'identifications_count': 69230,
   'journal_posts_count': 0,
   'activity_count': 76690,
   'species_count': 2300,
   'annotated_observations_count': 25036,
   'universal_search_rank': 7460,
   'roles': [],
   'site_id': 9,
   'icon_url': 'https://static.inaturalist.org/attachments/users/icons/3587396/medium.jpg?1603334730'},
  'vote_score': 0}]

In [28]:
plants

,species,life_history,plant_growth_form,fruit_colour,fruit_dehiscence,fruit_length_mm,fruit_type,leaf_length_mm,leaf_shape,plant_height_m,petiole_length_cm,seed_length_mm,sex_type,leaf_phenology
0,Abrotanella_nivigena,perennial,herb,purple,indehiscent,2.00,achene,14.0,linear narrowly_oblong,0.03,NaN,NaN,NaN,NaN
1,Abrotanella_scapigera,perennial,herb,brown,indehiscent,1.95,achene,25.0,narrowly_elliptical spathulate,0.10,NaN,NaN,NaN,NaN
2,Acacia_abbreviata,perennial,shrub,NaN,dehiscent,45.00,legume,14.5,linear narrowly_oblanceolate narrowly_oblong t...,0.45,NaN,3.75,NaN,NaN
3,Acacia_acinacea,perennial,shrub,NaN,NaN,NaN,legume,9.5,elliptical narrowly_oblong oblanceolate orbicu...,2.50,NaN,4.50,NaN,NaN
4,Acacia_aciphylla,perennial,shrub,NaN,NaN,90.00,legume,90.0,rhomboidal terete,1.50,NaN,2.75,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
846,Tinospora_esiangkara,perennial,climber_woody,NaN,NaN,6.00,NaN,NaN,NaN,NaN,NaN,NaN,dioecious,evergreen
847,Tinospora_smilacina,perennial,climber_woody,NaN,NaN,7.50,NaN,NaN,NaN,NaN,NaN,NaN,dioecious,evergreen
848,Trithuria_australis,annual,herb,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,monoecious,NaN
849,Triunia_robusta,perennial,shrub tree,NaN,NaN,25.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,evergreen
